# Parte II: efecto de agregación

En la primera parte de la práctica vimos cómo la escala de análisis tiene influencia sobre los resultados del mismo. En esta segunda parte vamos a ver como diferentes formas de agregar los datos, en una misma escala, también pueden tener una influencia sobre los resultados de un análisis.

Para esta segunda parte vamos a volver a trabajar con los datos de mezcla de usos de suelo en la Ciudad de México y vamos a tratar de estimar su efecto en la generación de viajes. Los datos de uso de suelo están calculados a partir del DENUE, mientras que la información sobre los viajes es de la Encuesta Origen Destino del 2007, por lo que la escala de análisis serán los distritos de tráfico de dicha encuesta.

Como siempre, el primer paso es leer los datos:

In [15]:
from geopandas import GeoDataFrame
datos = GeoDataFrame.from_file('data/distritos_variables.shp')
datos.head()

,comercio,cve_dist,entrada,geometry,gid,loop,ocio,pob,salida,viv
0,9614,034,201286,"POLYGON ((486947.2187500237 2148842.250066909,...",1,27627,95,2125687,200217,32618
1,2905,083,209264,"POLYGON ((478110.2187500393 2134697.250066519,...",2,76022,82,3563363,209859,42238
2,2944,113,65701,"POLYGON ((501006.0312499983 2148019.750066887,...",3,10611,72,3128496,66706,26003
3,2425,131,42385,"(POLYGON ((507974.718749986 2125105.250066254,...",4,8468,60,4842162,43106,44080
4,2849,063,128608,"POLYGON ((487190.7500000236 2140396.250066676,...",5,24472,30,4146145,128756,35981


El _shapefile_ que leimos tiene columnas para cada uno de los tipos de uso de suelo que nos interesan (vivienda, comercio, servicios y ocio), adicionalmente tiene tres columnas con información de los viajes:

* entrada: los viajes que terminan en el distrito y que empezaron en uno diferente
* salida: los viajes que empiezan en el distrito y terminan en uno distinto
* loop: los viajes que inician y terminan en el mismo distrito.

Como pueden recordar de la práctica anterior en que usamos estos datos, la mezcla de usos de suelo se mide utilizando el índice de  _entropía_, en esta ocasión vmos a calcularlo de acuerdo a la siguiente fórmula:

\begin{equation} 
E = \sum\limits_{j}{\frac{p_{j}*ln(p_{j})}{ln(J)}}
\end{equation}

Donde $p_{j}$ representa la proporción del $j-ésimo$ uso de suelo con respecto al total y $J$ es el número de usos de suelo.

In [16]:
import numpy as np
intensidad = datos['comercio'] + datos['viv'] + datos['ocio']
prop_comercio = datos['comercio'] / intensidad
prop_viv = datos['viv'] / intensidad
prop_ocio = datos['ocio'] / intensidad
entropia = (prop_comercio*np.log(prop_comercio) + prop_viv*np.log(prop_viv) + prop_ocio*np.log(prop_ocio))/np.log(3)
entropia.head()

0   -0.501674
1   -0.229016
2   -0.314356
3   -0.195138
4   -0.244285
dtype: float64

Lo que hicimos para calcular la _entropía_ es relativamente fácil. Primero creamos la variable intensidad, que es la suma de todos los usos de suelo y luego fuimos calculando cada una de las proporciones, finalmente las sumamos y las dividimos por el logaritmo natural del número de usus de suelo. Lo que tenemos ahora es una **Serie** que contiene los datos de entropía, ahora lo que necesitamos es unir esa serie a nuestros datos originales:

In [17]:
datos['entropia'] = entropia
datos.head()

,comercio,cve_dist,entrada,geometry,gid,loop,ocio,pob,salida,viv,entropia
0,9614,034,201286,"POLYGON ((486947.2187500237 2148842.250066909,...",1,27627,95,2125687,200217,32618,-0.501674
1,2905,083,209264,"POLYGON ((478110.2187500393 2134697.250066519,...",2,76022,82,3563363,209859,42238,-0.229016
2,2944,113,65701,"POLYGON ((501006.0312499983 2148019.750066887,...",3,10611,72,3128496,66706,26003,-0.314356
3,2425,131,42385,"(POLYGON ((507974.718749986 2125105.250066254,...",4,8468,60,4842162,43106,44080,-0.195138
4,2849,063,128608,"POLYGON ((487190.7500000236 2140396.250066676,...",5,24472,30,4146145,128756,35981,-0.244285


Ahora que ya tenemos todos los datos, probemos un modelo que nos trate de explicar, por ejemplo, la cantidad de viajes que terminan en cada distrito. Lo primero que vamos a hacer, es explorar la colinearidad de las variables que tenemos:

In [18]:
corr = datos[['ocio','comercio','viv','entropia']].corr()
w, v = np.linalg.eig(corr)
print w

[ 2.38401396  1.32379486  0.26654834  0.02564285]


Parece ser que, si utilizamos las variables de arriba, vamos a tener problemas de colinearidad, entonces, para seleccionar las variables, observemos la matriz de correlación:

In [19]:
print corr

              ocio  comercio       viv  entropia
ocio      1.000000  0.714724  0.586546 -0.364044
comercio  0.714724  1.000000  0.455756 -0.696061
viv       0.586546  0.455756  1.000000  0.272435
entropia -0.364044 -0.696061  0.272435  1.000000


Como puden ver, la entropía está bastante correlacionada con las demás variables, sobre todo con comercio y ocio, seleccionesmos entonces, por lo pronto, entropía y vivienda como variables explicativas.

Ahora, antes de correr nuestro modelo, vamos a normalizar las variables porque tiene escalas de variación muy diferentes y ya sabemos que eso nos puede traer problemas:

In [20]:
variables = datos[['entrada','viv','entropia']]
vars_norm = (variables - variables.mean())/variables.std()
print vars_norm.mean()
print vars_norm.std()

entrada     2.135044e-18
viv         1.978474e-16
entropia   -2.052838e-15
dtype: float64
entrada     1
viv         1
entropia    1
dtype: float64


Ahora que ya seleccionamos las varibles y las normalizamos, vamos a correr un primer modelo:

In [21]:
import statsmodels.formula.api as sm
model = sm.ols(formula="entrada ~ viv + entropia",data=vars_norm).fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                entrada   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     11.30
Date:                Tue, 05 Apr 2016   Prob (F-statistic):           2.65e-05
Time:                        13:02:31   Log-Likelihood:                -208.53
No. Observations:                 155   AIC:                             423.1
Df Residuals:                     152   BIC:                             432.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0051      0.075      0.068      0.9

### Ejercicio:
Especfique diferentes modelos para tratar de explicar las tres variables de viajes.

Ahora vamos a cambiar la escala del análisis agregando los distritos en unidades más grandes, para hacer esto, primero vamos a crear regionalizaciones al azar a parti de los datos:

In [22]:
import clusterpy
datos_cp = clusterpy.importArcData('data/distritos_variables')
datos_cp.cluster('random',['entrada','viv'],50,dissolve=0)

Loading data/distritos_variables.dbf
Loading data/distritos_variables.shp
Done
Getting variables
Variables successfully extracted
FINAL SOLUTION:  [35, 48, 17, 30, 44, 47, 33, 21, 8, 40, 10, 34, 0, 5, 7, 17, 45, 49, 36, 33, 18, 11, 43, 20, 46, 22, 2, 47, 44, 18, 36, 27, 27, 23, 34, 44, 30, 28, 14, 37, 24, 29, 41, 27, 9, 4, 10, 31, 43, 15, 39, 24, 33, 44, 17, 22, 9, 4, 16, 43, 15, 24, 21, 24, 21, 19, 17, 31, 4, 6, 45, 30, 34, 45, 8, 21, 33, 18, 13, 29, 48, 36, 11, 32, 45, 43, 40, 20, 37, 22, 42, 27, 4, 45, 0, 24, 28, 32, 6, 18, 24, 46, 36, 36, 32, 1, 0, 15, 12, 13, 38, 27, 7, 30, 21, 7, 17, 4, 24, 47, 17, 24, 19, 21, 47, 3, 37, 45, 45, 47, 0, 24, 47, 14, 32, 22, 22, 18, 15, 25, 36, 1, 8, 32, 32, 6, 12, 27, 4, 20, 21, 20, 12, 26, 27, 16]
FINAL OF:  192306132263.0
Done
Adding variables
Done


Recuerden que para ver las regiones resultantes pueden exportar los resultados como _shapefile_ usando `datos_cp.exportArcData('ruta/para/guardar')`. Por lo pronto omitiremos este paso y más bien vamos a unir el resultado de los clusters a los datos originales para poder hacer algunas operaciones.

El primer paso es saber en qué columna guardo el algoritmo los resultados de la regionalización, para esto, vamos a ver qué columnas tiene nuestro objeto:

In [24]:
datos_cp.fieldNames

['ID',
 'comercio',
 'cve_dist',
 'entrada',
 'gid',
 'loop',
 'ocio',
 'pob',
 'salida',
 'viv',
 'random_20160405130236']

Como pueden ver, el algoritmo le pone un nombre al azar a la columna donde va a guardar los resultados, `random_20160405130236`, en este caso.

Ahora lo que vamos a hacer es extraer sólo el identificador de distrito y el identificador de región, que es lo que necesitamos para hacer un _merge_ con nuestros datros originales:

In [25]:
resultado = datos_cp.getVars(['cve_dist','random_20160405130236'])
print resultado

Getting variables
Variables successfully extracted
{0: ['034', 35], 1: ['083', 48], 2: ['113', 17], 3: ['131', 30], 4: ['063', 44], 5: ['135', 47], 6: ['056', 33], 7: ['024', 21], 8: ['060', 8], 9: ['071', 40], 10: ['156', 10], 11: ['149', 34], 12: ['107', 0], 13: ['001', 5], 14: ['098', 7], 15: ['114', 17], 16: ['074', 45], 17: ['032', 49], 18: ['103', 36], 19: ['117', 33], 20: ['061', 18], 21: ['145', 11], 22: ['048', 43], 23: ['104', 20], 24: ['029', 46], 25: ['055', 22], 26: ['150', 2], 27: ['091', 47], 28: ['053', 44], 29: ['079', 18], 30: ['099', 36], 31: ['086', 27], 32: ['009', 27], 33: ['140', 23], 34: ['146', 34], 35: ['037', 44], 36: ['084', 30], 37: ['039', 28], 38: ['022', 14], 39: ['080', 37], 40: ['094', 24], 41: ['151', 29], 42: ['018', 41], 43: ['049', 27], 44: ['010', 9], 45: ['043', 4], 46: ['153', 10], 47: ['096', 31], 48: ['008', 43], 49: ['072', 15], 50: ['155', 39], 51: ['015', 24], 52: ['126', 33], 53: ['052', 44], 54: ['144', 17], 55: ['118', 22], 56: ['092', 9

El resultado es un _diccionario_, es decir, un conjunto de parejas _llave : valor_, en este caso las llaves son el id de los polígonos y los valores son la clave del distrito y el identificador de la región. Como queremos unir estos resultados con nuestros datos originales, necesitamos convertirlos en un **DataFrame**, afortunadamete esto es relativamente fácil:

In [28]:
import pandas as pd
df = pd.DataFrame(resultado)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
0,034,083,113,131,063,135,056,024,060,071,...,137,070,040,100,035,106,138,019,050,148
1,35,48,17,30,44,47,33,21,8,40,...,12,27,4,20,21,20,12,26,27,16


Casi lo logramos, el único problema es que los datos están "al revés", es decir renglones y columnas están intercambiados. Lo que necesitamos es trasponerlos:º

In [29]:
df = df.transpose()
df.head()

,0,1
0,034,35
1,083,48
2,113,17
3,131,30
4,063,44


Ahora pongámosle nombre a las columnas, la primera es el identificador de distrito y la segunda el de región:

In [31]:
df.columns=['cve_dist','id_region']
df.head()

,cve_dist,id_region
0,034,35
1,083,48
2,113,17
3,131,30
4,063,44


Ahora sí podemos hacer un _merge_ con los datos originales:

In [32]:
region_1 = datos.merge(df,how='inner',on='cve_dist')
region_1.head()

,comercio,cve_dist,entrada,geometry,gid,loop,ocio,pob,salida,viv,entropia,id_region
0,9614,034,201286,"POLYGON ((486947.2187500237 2148842.250066909,...",1,27627,95,2125687,200217,32618,-0.501674,35
1,2905,083,209264,"POLYGON ((478110.2187500393 2134697.250066519,...",2,76022,82,3563363,209859,42238,-0.229016,48
2,2944,113,65701,"POLYGON ((501006.0312499983 2148019.750066887,...",3,10611,72,3128496,66706,26003,-0.314356,17
3,2425,131,42385,"(POLYGON ((507974.718749986 2125105.250066254,...",4,8468,60,4842162,43106,44080,-0.195138,30
4,2849,063,128608,"POLYGON ((487190.7500000236 2140396.250066676,...",5,24472,30,4146145,128756,35981,-0.244285,44


Ahora ya tenemos casi todo lo que necesitamos para correr un nuevo modelo, esta vez sobre las variables agregadas en regiones, lo único que necesitamos es, justamente, calcular las variables agregadas, para esto, vamos a hacer un "gropup by":

In [35]:
agregados = region_1.groupby(by='id_region').sum()
agregados.head()

,comercio,entrada,gid,loop,ocio,pob,salida,viv,entropia
id_region,,,,,,,,,
0,12876,448433,346,228713,307,23204604,450009,208926,-0.841356
1,15498,276225,248,14118,150,3346899,275092,57157,-0.841699
2,7907,123731,27,194316,183,12768687,123594,124519,-0.215108
3,1638,57328,126,8536,23,1535879,57568,19774,-0.253278
4,14381,1165711,533,148210,415,11466251,1158579,195706,-1.441266


Aquí simplemente agrupamos los datos por su id de región y calculamos la suma de las variables sobre cada grupo.

Sólo hay un problema, la entropía quedo calculada como la suma de las entropías individuales y eso no nos sirve, necesitamos volver a calcularla:

In [37]:
intensidad = agregados['comercio'] + agregados['viv'] + agregados['ocio']
prop_comercio = agregados['comercio'] / intensidad
prop_viv = agregados['viv'] / intensidad
prop_ocio = agregados['ocio'] / intensidad
entropia = (prop_comercio*np.log(prop_comercio) + prop_viv*np.log(prop_viv) + prop_ocio*np.log(prop_ocio))/np.log(3)
agregados.ix[:,'entropia']= entropia
agregados.head()

,comercio,entrada,gid,loop,ocio,pob,salida,viv,entropia
id_region,,,,,,,,,
0,12876,448433,346,228713,307,23204604,450009,208926,-0.210947
1,15498,276225,248,14118,150,3346899,275092,57157,-0.484285
2,7907,123731,27,194316,183,12768687,123594,124519,-0.215108
3,1638,57328,126,8536,23,1535879,57568,19774,-0.253278
4,14381,1165711,533,148210,415,11466251,1158579,195706,-0.239735


Ahora sí, podemos correr el mismo modelo de antes pero sobre nuestros datos agregados:

In [38]:
model = sm.ols(formula="entrada ~ viv + entropia",data=agregados).fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                entrada   R-squared:                       0.479
Model:                            OLS   Adj. R-squared:                  0.457
Method:                 Least Squares   F-statistic:                     21.17
Date:                Tue, 05 Apr 2016   Prob (F-statistic):           3.03e-07
Time:                        13:49:03   Log-Likelihood:                -661.50
No. Observations:                  49   AIC:                             1329.
Df Residuals:                      46   BIC:                             1335.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept  -1.015e+05   1.12e+05     -0.903      0.3